# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute_target import ComputeTargetException
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#setting workspace from the Azure configuration file
ws = Workspace.from_config()
experiment_name = 'ml-exp'

experiment=Experiment(ws, experiment_name)
dataset = Dataset.get_by_name(ws, name='heart_disease')
data = dataset.to_pandas_dataframe()

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
#name of the compute cluster
cpu_cluster_name = "mycluster"

# Verify that cluster does not exist already (good practice extracted by Microsoft official documentation: https://docs.microsoft.com/)
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
#creating compute (cpu) cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
    '--C': uniform(0.01, 0.1),
    '--max_iter': choice(10, 20, 50, 100, 250)
})


# Setup environment for your training run

estimator = SKLearn(source_directory='.',#folder for experiment configuration files.
              entry_script=  'train.py', #relative path to the file used to start training (training script)
              compute_target=cpu_cluster)
#TODO: Create your estimator and hyperdrive config

hyperdrive_config = HyperDriveConfig(
    estimator=estimator,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=100)

In [ ]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Getting the best model, saving and displaying all properties.

#finding the best model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']

print('Best Run Id: ', best_run.id)
print('\nAccuracy of best run :', best_run_metrics['Accuracy'])
print('\nHyperparam values of best run:',parameter_values['arguments'])

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='best_model_HD', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

